# Proxy-Tuning with Llama 7

## Runs in Single-GPU Setting

In [1]:
%load_ext watermark
%watermark -p torch,transformers,bitsandbytes

torch       : 2.1.2+cu121
transformers: 4.37.1
bitsandbytes: 0.42.0



Make sure that you use a machine with a GPU.

In [2]:
import torch

if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")
    if num_gpus < 1:
        raise Exception("At least 1 GPU is required for this setup.")
else:
    raise Exception("No GPUs found, make sure a GPU is available")


Number of GPUs available: 1


### Loading the Models

In this section, we are applying proxy-tuning to boost the performance of a Llama 7B base model to that of a Llama 13B Chat model. (We focus on the 13B model because it conveniently fits into the RAM of an affordable GPU like A10G, but the same methodology could also be applied to 70B models for even more impressive results.)

First, we download and load each model to ensure the models are available:

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os


model_names = [
    "meta-llama/Llama-2-7b-hf",
    "meta-llama/Llama-2-7b-chat-hf",
    "meta-llama/Llama-2-13b-hf",
    "meta-llama/Llama-2-13b-chat-hf",
]

device = "cuda"

for i, model_name in enumerate(model_names):
    model_dir = os.path.join("models", model_name)
    print("Loading:", model_dir)

    if not os.path.exists(model_dir): # use HF token to download models if they don't exist on disk yet
        hf_token = "your access token"

        if "7b" in model_name:
            model = AutoModelForCausalLM.from_pretrained(
                model_name, token=hf_token, cache_dir="models", device_map=device,
                attn_implementation="flash_attention_2", torch_dtype=torch.bfloat16
            )
        else:
            model = AutoModelForCausalLM.from_pretrained(
                model_name, token=hf_token, cache_dir="models", device_map=device,
                attn_implementation="flash_attention_2", torch_dtype=torch.bfloat16, load_in_4bit=True
            )
        del model
        tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token, cache_dir="models")
        del tokenizer
    else:
        if "7b" in model_name:
            model = AutoModelForCausalLM.from_pretrained(
                model_dir, device_map=device, attn_implementation="flash_attention_2", 
                torch_dtype=torch.bfloat16
            )
        else:
            model = AutoModelForCausalLM.from_pretrained(
                model_dir, device_map=device, attn_implementation="flash_attention_2",
                torch_dtype=torch.bfloat16, load_in_4bit=True
            )
        del model
        tokenizer = AutoTokenizer.from_pretrained(model_dir)
        del tokenizer


print("Models have been downloaded and are available.")

Loading: models/meta-llama/Llama-2-7b-hf


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading: models/meta-llama/Llama-2-7b-chat-hf


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading: models/meta-llama/Llama-2-13b-hf


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading: models/meta-llama/Llama-2-13b-chat-hf


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Models have been downloaded and are available.


### Generating Text with Unmodified Models

As a reminder, note that the "Llama-2-13b-chat-hf" is a control experiment. We aim to use a smaller available "Llama-2-7b-chat-hf" model to achieve the same performance when we apply it on top of the "Llama-2-13b-hf" base model.

Next, let's check the outputs that these models produce without applying proxy-tuning, yet:

In [4]:
def load_model_and_tokenizer(model_dir):

    hf_token = "your access token"

    if "7b" in model_name:
        model = AutoModelForCausalLM.from_pretrained(
            model_name, token=hf_token, cache_dir="models", device_map=device,
            attn_implementation="flash_attention_2", torch_dtype=torch.bfloat16
        )
    else:
        model = AutoModelForCausalLM.from_pretrained(
            model_name, token=hf_token, cache_dir="models", device_map=device,
            attn_implementation="flash_attention_2", torch_dtype=torch.bfloat16, load_in_4bit=True
        )
    tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token, cache_dir="models")
    return model, tokenizer


def generate_text(model, tokenizer, input_text, max_length):
    device = next(model.parameters()).device
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    generated_tokens = []

    with torch.no_grad():
        for _ in range(max_length):
            
            outputs = model(input_ids)
            logits = outputs.logits
            predictions = torch.softmax(logits[:, -1, :], dim=-1)
            next_token_id = torch.argmax(predictions).unsqueeze(0)
            generated_tokens.append(next_token_id.item())
            
            # Append the new token to the input sequence for the next iteration
            input_ids = torch.cat([input_ids, next_token_id.unsqueeze(0)], dim=1)

            if next_token_id.item() == tokenizer.eos_token_id:
                break

    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return generated_text


model_names = [
    "meta-llama/Llama-2-7b-hf",
    "meta-llama/Llama-2-7b-chat-hf",
    "meta-llama/Llama-2-13b-hf",
    "meta-llama/Llama-2-13b-chat-hf",
]

for model_name in model_names:
    input_text = ("If I have 5 apples and eat 2, but then find 3 more on my way home, how many do I have?")

    print("Loading", model_name)
    model, tokenizer = load_model_and_tokenizer(model_name)
    
    print(f"{50*'='}\n\nAsking {model_name}:")
    generated = generate_text(model, tokenizer, input_text, max_length=60)
    print(generated, f"\nLength: {len((input_text+generated).split(' '))}")

    del model
    del tokenizer

Loading meta-llama/Llama-2-7b-hf


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


Asking meta-llama/Llama-2-7b-hf:

I have 5 apples.
I eat 2.
I find 3 more.
I have 10 apples.
I have 5 apples.
I eat 2.
I find 3 more.
I have 10 apples. 
Length: 44
Loading meta-llama/Llama-2-7b-chat-hf


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


Asking meta-llama/Llama-2-7b-chat-hf:


Answer: You have 5 apples in total.

Explanation:

When you started your journey, you had 5 apples.

When you ate 2 apples, you had 5 - 2 = 3 apples left.
 
Length: 49
Loading meta-llama/Llama-2-13b-hf


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]


Asking meta-llama/Llama-2-13b-hf:

If I have 5 apples and eat 2, but then find 3 more on my way home, how many do I have?
I have 5 apples and eat 2, but then find 3 more on my way home, how many do I have?
 
Length: 63
Loading meta-llama/Llama-2-13b-chat-hf


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]


Asking meta-llama/Llama-2-13b-chat-hf:


You start with 5 apples and eat 2, so you have 5 - 2 = 3 apples left.
Then, you find 3 more apples on your way home, so you have 3 + 3 = 6 apples in total. 
Length: 59


As we can see, only the "Llama-2-13b-chat-hf" model (our control experiment) can achieve the correct response. 

### Proxy-tune the 13B Base model using Llama 2 7B Chat

In [5]:
import sys
import os
import contextlib
from tqdm import tqdm


@contextlib.contextmanager
def suppress_output():
    """Context manager to suppress stdout and stderr."""
    new_stdout, new_stderr = os.devnull, os.devnull
    old_stdout, old_stderr = sys.stdout, sys.stderr
    sys.stdout, sys.stderr = open(new_stdout, 'w'), open(new_stderr, 'w')
    try:
        yield
    finally:
        sys.stdout.close()
        sys.stderr.close()
        sys.stdout, sys.stderr = old_stdout, old_stderr


def generate_proxy_tuning(model_base_name, model_tuned_name, model_target_name, input_text, max_length):
    # Load the tokenizer once as we assume it's the same for all models
    _, tokenizer = load_model_and_tokenizer(model_base_name)

    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    generated_tokens = []

    with torch.no_grad():
        for _ in tqdm(range(max_length), desc="Generating tokens"):
            # Suppress output during model loading
            with suppress_output():
                model_base, _ = load_model_and_tokenizer(model_base_name)
            logits_base = model_base(input_ids).logits
            del model_base

            with suppress_output():
                model_tuned, _ = load_model_and_tokenizer(model_tuned_name)
            logits_tuned = model_tuned(input_ids).logits
            del model_tuned

            with suppress_output():
                model_target, _ = load_model_and_tokenizer(model_target_name)
            logits_target = model_target(input_ids).logits
            del model_target

            # Calculate final logits
            logits = logits_target + (logits_tuned - logits_base)

            # Generate next token
            predictions = torch.softmax(logits[:, -1, :], dim=-1)
            next_token_id = torch.argmax(predictions).unsqueeze(0)
            generated_tokens.append(next_token_id.item())

            # Append the new token to the input sequence for the next iteration
            input_ids = torch.cat([input_ids, next_token_id.unsqueeze(0)], dim=1)

            # Break if end of sequence token is generated
            if next_token_id.item() == tokenizer.eos_token_id:
                break

    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return generated_text


generated = generate_proxy_tuning(
    model_base_name="meta-llama/Llama-2-7b-hf",
    model_tuned_name="meta-llama/Llama-2-7b-chat-hf",
    model_target_name="meta-llama/Llama-2-13b-hf",
    input_text=input_text,
    max_length=100
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Generating tokens:   0%|          | 0/100 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

...

Generating tokens:  60%|██████    | 60/100 [26:59<18:04, 27.11s/it]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Generating tokens:  60%|██████    | 60/100 [27:26<18:17, 27.45s/it]


As we can see, the `"Llama-2-13b-hf"` model, which previously couldn't answer the question correctly, can now produce an answer similar to the answer created via `"Llama-2-13b-chat-hf"`. And we were able to achieve this without changing the weights of that `"Llama-2-13b-hf"` model!

In [6]:
print(generated)



You start with 5 apples and eat 2, so you have 5 - 2 = 3 apples left.
Then, you find 3 more apples on your way home, so you have 3 + 3 = 6 apples in total.
